In [1]:
from resources.runmanager import RunManager
from resources.runbuilder import RunBuilder
from collections import OrderedDict
import torch
from collections import namedtuple
from itertools import product
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F

In [2]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)

        self.fc1 = nn.Linear(in_features=12 * 4 * 4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)

    def forward(self, t):
        # (1) input layer
        t = t

        # (2) hidden conv layer
        t = self.conv1(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        # (3) hidden conv layer
        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        # (4) hidden linear layer
        t = t.reshape(-1, 12 * 4 * 4)
        t = self.fc1(t)
        t = F.relu(t)

        # (5) hidden linear layer
        t = self.fc2(t)
        t = F.relu(t)

        # (6) output layer
        t = self.out(t)
        #t = F.softmax(t, dim=1)

        return t

In [3]:
train_set = torchvision.datasets.FashionMNIST(
    root='./data'
    ,train=True
    ,download=True
    ,transform=transforms.Compose([
        transforms.ToTensor()
    ])
)

In [ ]:
params = OrderedDict(
    lr=[.01, .001, .0001, .00001]
    , batch_size=[1000, 2000]
    , shuffle=[True, False]
    , device = ['cpu', 'cuda']
)
m = RunManager()

for run in RunBuilder.get_runs(params):
    device=torch.device(run.device)
    network=Network().to(device)
    loader=torch.utils.data.DataLoader(train_set, batch_size=run.batch_size, shuffle=run.shuffle)
    optimizer=optim.Adam(network.parameters(), lr=run.lr)

    m.begin_run(run, network, loader)

    for epoch in range(5):
        m.begin_epoch()
        for batch in loader:
            images = batch[0].to(device)
            labels = batch[1].to(device)
            preds = network(images) # Pass Batch
            loss = F.cross_entropy(preds, labels) # Calculate Loss

            optimizer.zero_grad()
            loss.backward() # Calculate Gradients
            optimizer.step() # Update Weights

            m.track_loss(loss, batch)
            m.track_num_correct(preds,labels)
        m.end_epoch()
    m.end_run()
m.save('results')


,run,epoch,loss,accuracy,epoch duration,run duration,lr,batch_size,shuffle,device
0,1,1,0.959026,0.644800,7.093827,7.819344,0.01,1000,True,cpu
1,1,2,0.504389,0.810550,6.883558,14.839220,0.01,1000,True,cpu
2,1,3,0.420672,0.846200,6.926723,21.864103,0.01,1000,True,cpu
3,1,4,0.382504,0.859483,6.882400,28.879078,0.01,1000,True,cpu
4,1,5,0.349445,0.872067,7.025353,36.028194,0.01,1000,True,cpu
